# [LVV-T2214] - MTMount_Elevation_Changes_with_MTAOS_Aberrations

This notebook was originally written by Bo Xin in the [lsst-ts/ts_notebooks] repository.  
It is a modified version with updated commands and simplified steps.

**Make sure you run this notebook on TTS before running at the summit.**

[lsst-ts/ts_notebooks]: https://github.com/lsst-ts/ts_notebooks/blob/develop/bxin/aos2comp/aos2comp.ipynb
[LVV-T2214]: https://jira.lsstcorp.org/secure/Tests.jspa#/testCase/LVV-T2214

## Requirements

This notebooks require some extra repositories to be installed locally so it can grab some constants and some look-up tables.  
Here is a list of which repos are these:

* [lsst-ts/ts_cRIOpy]

[lsst-ts/ts_cRIOpy]: https://github.com/lsst-ts/ts_cRIOpy 
[lsst-ts/ts_m1m3support]: https://github.com/lsst-ts/ts_m1m3support

To install them, open a terminal and run the following commands:

```
git clone https://github.com/lsst-ts/ts_cRIOpy $HOME/notebooks/lsst-ts/ts_cRIOpy
git clone https://github.com/lsst-ts/ts_m1m3support.git $HOME/notebooks/lsst-ts/ts_m1m3support
git clone https://github.com/lsst-ts/ts_config_mttcs $HOME/notebooks/lsst-ts/ts_config_mttcs
git clone https://github.com/lsst-sitcom/M2_FEA $HOME/notebooks/lsst-sitcom/M2_FEA
```

And add these lines to your `$HOME/notebooks/.user_setup` file: 

```
setup -j notebooks_vandv -r ~/notebooks/lsst-sitcom/notebooks_vandv
setup -j ts_cRIOpy -r $HOME/notebooks/lsst-ts/ts_cRIOpy
```

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from lsst.sitcom import vandv

exec_info = vandv.ExecutionInfo()
print(exec_info)

---
## Setup Notebook for Test

- Import all libraries
- Get the remotes ready

In [ ]:
test_execution = "LVV-D220620"

In [ ]:
import asyncio
import os
import yaml

import astropy.units as u
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import time
import warnings

from astropy.time import Time, TimeDelta
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5
from datetime import datetime, timedelta

from lsst_efd_client import EfdClient
from lsst.ts import utils, salobj
from lsst.ts.cRIOpy import M1M3FATable
from lsst.ts.observatory.control.maintel.mtcs import MTCS, MTCSUsages
from lsst.ts.observatory.control import RotType

In [ ]:
c = EfdClient("tucson_teststand_efd")

In [ ]:
df = await c.select_top_n(
    "lsst.sal.MTM1M3.logevent_appliedElevationForces", 
    fields="*", 
    num=1, 
    time_cut=Time("2022-06-15T23:59", format="isot", scale="utc") 
)

print(df)

In [ ]:
print(os.environ.get("LSST_DDS_DOMAIN_ID"))

In [ ]:
print(os.environ["OSPL_URI"])
print(os.environ["LSST_DDS_PARTITION_PREFIX"])
print(os.environ.get("LSST_DDS_DOMAIN_ID"))

In [ ]:
logging.basicConfig(format="%(name)s:%(message)s", level=logging.DEBUG)

In [ ]:
log = logging.getLogger("setup")
log.level = logging.DEBUG

In [ ]:
domain = salobj.Domain()

In [ ]:
mtcs = MTCS(domain=domain, log=log)
mtcs.set_rem_loglevel(40)

In [ ]:
await mtcs.start_task

In [ ]:
warnings.filterwarnings("ignore", message=".*DDS read queue.*")

## Helper Functions

In [ ]:
async def get_data_from_efd(location, start, end):
    """
    Retrieves data relevant for analysis.
    
    Parameters
    ----------
    location : str
        In which Test Stand are you running this test?
    start : str or datetime
        Start of the time-window.
    end : str or datetime 
        End of the time-window.
    """
    if location == "summit":
        client = EfdClient("summit_efd")
    elif location == "tucson":
        client = EfdClient("tucson_teststand_efd")
    else:
        raise ValueError(
            "Location does not match any valid options {summit|tucson}"
        )
    
    _df_mount_el = await client.select_time_series(
        "lsst.sal.MTMount.elevation", 
        fields="actualPosition", 
        start=start, 
        end=end,
    )
    
    _df_m1m3_101 = await client.select_time_series(
        "lsst.sal.MTM1M3.forceActuatorData",
        fields="zForce101",
        start=start, 
        end=end,
    )
        
    # M2 B1?
    _df_m2 = await client.select_time_series(
        "lsst.sal.MTM2.axialForce",
        fields=[
            "applied0",
            "lutGravity0",
            "measured0"
        ],
        start=start, 
        end=end,
    )
    
    # CamHex Y position
    _df_camhex = await client.select_time_series(
        "lsst.sal.MTHexapod.application",
        fields=[
            "position1"
        ],
        index=1,
        start=start, 
        end=end,
    )
    
    # M2Hex Y position
    _df_m2hex = await client.select_time_series(
        "lsst.sal.MTHexapod.application",
        fields=[
            "position1"
        ],
        index=2,
        start=start, 
        end=end,
    )

    # Rename columns
    _df_mount_el.rename(columns={"actualPosition": "mount_el"}, inplace=True)
    _df_m1m3_101.rename(columns={"zForce101": "m1m3_z101"}, inplace=True)
    _df_m2.rename(columns={"applied0": "m2b1_applied", "lutGravity0": "m2b1_gravLut", "measured0": "m2b1_measured"}, inplace=True)
    _df_camhex.rename(columns={"position1": "camhex_y"}, inplace=True)
    _df_m2hex.rename(columns={"position1": "m2hex_y"}, inplace=True)

    
    # Join dataframes
    _df = _df_mount_el
    
    _df = pd.merge(
        _df, 
        _df_m1m3_101, 
        left_index=True, 
        right_index=True, 
        how="outer"
    )
         
    _df = pd.merge(
        _df, 
        _df_m2, 
        left_index=True, 
        right_index=True, 
        how="outer"
    )
    
    _df = pd.merge(
        _df, 
        _df_camhex, 
        left_index=True, 
        right_index=True, 
        how="outer"
    )

    _df = pd.merge(
        _df, 
        _df_m2hex, 
        left_index=True, 
        right_index=True, 
        how="outer"
    )
    
    return _df

In [ ]:
async def moveMountInElevationSteps(target_el, azimuth=0, step_size=0.25, time_sleep=1):
    """Move the mount from the current elevation angle to the target elevation angle 
    in steps to avoid any issues whe M1M3 and/or M2 are running with the LUT using the 
    Mount instead of the inclinometer.
    
    This function will actually calculate the number of steps using the ceiling
    in order to make sure that we move carefully. 
    
    Parameters
    ----------
    target_el : float
        Target elevation angle in degrees
    azimuth : float
        Azimuth angle in degres (default)
    step_size : float
        Step elevation size in degrees (default: 0.25)
    time_sleep : float
        Sleep time between movements (default: 1)
        
    Returns
    -------
    azimuth : float
        Current azimuth
    elevation : float
        Current elevation
    """
    current_el = mtcs.rem.mtmount.tel_elevation.get().actualPosition
    n_steps = int(np.ceil(np.abs(current_el - target_el) / step_size))

    for el in np.linspace(current_el, target_el, n_steps):
        print(f"Moving elevation to {el:.2f} deg")
        await mtcs.rem.mtmount.cmd_moveToTarget.set_start(azimuth=azimuth, elevation=el)
        time.sleep(time_sleep)
        
    return azimuth, el

## Start all the components, and put them in an enabled state.

Depending on which test cycle this is being executed in, each component is either a hardware component or a simulator:

- [ ] M1M3
- [ ] M2 
- [ ] M2 Hexapod
- [ ] Camera Hexapod
- [ ] MTMount
- [ ] MTRotator
- [ ] MTPtg

The best/simplest way of doing this is running the notebook that executes the [LVV-T2344] test case.  
This notebook puts all the components in a ENABLED state.

[LVV-T2344]: https://github.com/lsst-sitcom/notebooks_vandv/blob/develop/notebooks/proj_sys_eng/sitcom_integration/l3_system_integ/LVV-T2344-Startup_mt_on_level3.ipynb

In [ ]:
# Verify that all the components have heartbeats
await mtcs.assert_liveliness()

In [ ]:
# Verify that all the components are enabled
await mtcs.assert_all_enabled()

First we slew and track at 89.5 degrees elevation


---
## Check elevation motion is smooth and no abrupt changes occur (Otherwise it faults M1M3) 

Do the following slew sequence, watch chronograph to make sure there is no abrupt change in elevation angle (otherwise it faults M1M3).

Do 4 slews in sequence. For each slew, track for 39s, simulating a visit.
Note: the 4 slews need to correspond to elevation angle between 86.5 deg and 82 deg. And, additional margin is needed due to tracking.

Move to zenith at the end (so that we can start m1m3 with LUT in mount telemetry mode)

In [ ]:
await mtcs.stop_tracking()

In [ ]:
warnings.simplefilter("ignore")
await moveMountInElevationSteps(90., azimuth=190.5)

In [ ]:
t_start = Time(Time.now(), format='fits', scale='utc') 
print(f"Gathering data - Elevation motion test - Start time: {t_start}")

az, el = await moveMountInElevationSteps(85.4, azimuth=190.5)

print('Tracking first source')
target_1 = mtcs.radec_from_azel(az=az, el=el)
await mtcs.slew_icrs(ra=target_1.ra, dec=target_1.dec, rot_type=RotType.Physical, rot=0)

time.sleep(39.)
await mtcs.stop_tracking()

In [ ]:
az, el = await moveMountInElevationSteps(84.4, azimuth=190.5)

print('Tracking second source')
target = mtcs.radec_from_azel(az=az, el=el)
await mtcs.slew_icrs(ra=target.ra, dec=target.dec, rot_type=RotType.Physical, rot=0)

time.sleep(39.)
await mtcs.stop_tracking()

In [ ]:
az, el = await moveMountInElevationSteps(83.4, azimuth=190.5)

print('Tracking second source')
target = mtcs.radec_from_azel(az=az, el=el)
await mtcs.slew_icrs(ra=target.ra, dec=target.dec, rot_type=RotType.Physical, rot=0)

time.sleep(39.)
await mtcs.stop_tracking()

In [ ]:
az, el = await moveMountInElevationSteps(82.4, azimuth=190.5)

print('Tracking second source')
target = mtcs.radec_from_azel(az=az, el=el)
await mtcs.slew_icrs(ra=target.ra, dec=target.dec, rot_type=RotType.Physical, rot=0)

time.sleep(39.)
await mtcs.stop_tracking()

In [ ]:
t_end = Time(Time.now(), format='fits', scale='utc')

print(f"Gathering data - Elevation motion test - End time: {t_end}")

In [ ]:
client = EfdClient("summit_efd")

In [ ]:
# Plot mount elevation as a function of time
# Query telemetry
dfm_elevation = await client.select_time_series('lsst.sal.MTMount.elevation', 'actualPosition', t_start.utc, t_end.utc)

In [ ]:
# Plot 
fig, axs = plt.subplots(figsize=(12, 6), sharex=True)

axs.plot(dfm_elevation["actualPosition"].dropna(), "k", label="Mount Elevation")
axs.set_ylabel("Mount El\n[deg]")

fig.suptitle(f"{test_execution} - Elevation vs Time")
fig.tight_layout(h_pad=0.3)
fig.patch.set_facecolor('white')   

fig.savefig(f"plots/{test_execution}_mount_el_vs_time.png")
plt.show()

---
## Get M1M3 Ready

- Raise the mirror,
- Turn on Balance Forces,
- Clear forces

Need to have M1M3 LUT use mount telemetry.

In [ ]:
await mtcs.set_state(salobj.State.STANDBY, components=["mtm1m3"])

In [ ]:
await mtcs.set_state(
    state=salobj.State.ENABLED,
    components=["mtm1m3"], 
    overrides = {"mtm1m3": 'Default'}
    )

In [ ]:
# If M1M3 is not raised yet, use this command to raise it.
await mtcs.raise_m1m3()

In [ ]:
# Enables M1M3 Force Balance system using the hardpoints
await mtcs.enable_m1m3_balance_system()

In [ ]:
# Resets the Aberration Forces and the Active Optics Forces
await mtcs.reset_m1m3_forces()

In [ ]:
# M1M3 LUT use mount telemetry
# todo: how to do that? 

## Get M2 Ready
- Turn on Force Balance system
- Clear forces

In [ ]:
# Enabled M2 Force Balance system 
await mtcs.enable_m2_balance_system()

In [ ]:
# Resets the Active Optics Forces
await mtcs.reset_m2_forces()

In [ ]:
# Need to have M2 LUT use mount telemetry
# todo: how to do that?

## Get CamHex Ready
- Check config 
- Make sure LUT is on, and has valid inputs
- Make sure hex is at LUT position

In [ ]:
# Check the configuration
await vandv.hexapod.get_hexapod_configuration(mtcs.rem.mthexapod_1)

In [ ]:
# Enable compensation mode for CamHex
await mtcs.enable_compensation_mode("mthexapod_1")

In [ ]:
# Reset the Camera Hexapod position
await mtcs.reset_camera_hexapod_position()

In [ ]:
# After resetting the Camera Hexapod position, we want to make sure that 
# the compesation and non-compensation values are the same.
await vandv.hexapod.print_hexapod_uncompensation_values(mtcs.rem.mthexapod_1)
await vandv.hexapod.print_hexapod_compensation_values(mtcs.rem.mthexapod_1)

In [ ]:
# Need to have CamHex LUT use mount telemetry
await vandv.hexapod.check_hexapod_lut(mtcs.rem.mthexapod_1)

## Get M2Hex Ready

- Check config 
- Make sure LUT is on, and has valid inputs
- Make sure M2Hex is at LUT position

In [ ]:
# Check the configuration
await vandv.hexapod.get_hexapod_configuration(mtcs.rem.mthexapod_2)

In [ ]:
# Enable compensation mode for M2Hex
await mtcs.enable_compensation_mode("mthexapod_2")

In [ ]:
# Reset the M2 Hexapod position
await mtcs.reset_m2_hexapod_position()

In [ ]:
# After resetting the Camera Hexapod position, we want to make sure that 
# the compesation and non-compensation values are the same.
await vandv.hexapod.print_hexapod_uncompensation_values(mtcs.rem.mthexapod_2)
await vandv.hexapod.print_hexapod_compensation_values(mtcs.rem.mthexapod_2)

In [ ]:
# Need to have CamHex LUT use mount telemetry
await vandv.hexapod.check_hexapod_lut(mtcs.rem.mthexapod_1)

## Gather Data - Without Aberrations
Do 4 slews in sequence. For each slew, track for 39s, simulating a visit.
Note: the 4 slews need to correspond to elevation angle between 86.5 deg and 82 deg. And, additional margin is needed due to tracking.

In [ ]:
# We start at 85.4 deg because the track fails at 85.5 deg
target_elevations = [85.4, 84.4, 83.4, 82.4]

t_start_without_aberration = Time.now()
t_start_without_aberration.format = "isot"
print(f"Gathering data - without Aberrations - Start time: {t_start_without_aberration}")

for _el in target_elevations:
    az, el = await moveMountInElevationSteps(_el, azimuth=190.5)

    target = mtcs.radec_from_azel(az=az, el=el)
    await mtcs.slew_icrs(ra=target.ra, dec=target.dec, rot_type=RotType.Physical, rot=0)

    # 39. seconds is the exposure time (2 * 15 sec) plus readout
    time.sleep(39.)
    await mtcs.stop_tracking()
    
t_end_without_aberration = Time.now()
t_end_without_aberration.format = "isot"
print(f"Gathering data - without Aberrations - End time: {t_end_without_aberration}")

### Plot Optics vs Time

Plot the following as a function of time during the above process:

- mount elevation
- m1m3 actuator 101 z force
- m2 actuator B1 force
- camera hex y position
- m2 hex y position

In [ ]:
print(f"Gathering data - without Aberrations - Start time: {t_start_without_aberration}")
print(f"Gathering data - without Aberrations - End time: {t_end_without_aberration}")

In [ ]:
# Query telemetry
df = await get_data_from_efd(
    exec_info.loc, 
    t_start_without_aberration, 
    t_end_without_aberration)

In [ ]:
fig, axs = plt.subplots(figsize=(10, 10), nrows=4, sharex=True)

axs[0].plot(df["mount_el"].dropna(), "k", label="Mount Elevation")
axs[0].set_ylabel("Mount El\n[deg]")

axs[1].plot(df["m1m3_z101"].dropna(), "C0o-", label="z101")
axs[1].set_ylabel("M1M3 Forces\n[--]")

# axs[2].plot(df["m2b1_applied"].dropna(), "C1-", label="applied")
# axs[2].plot(df["m2b1_gravLut"].dropna(), "C2-", label="Gravity LUT")
axs[2].plot(df["m2b1_measured"].dropna(), "C3-", label="Measured")
axs[2].set_ylabel("M2 Forces\n[--]")

axs[3].plot(df["camhex_y"].dropna(), "C3x-", label="CamHex Y")
axs[3].plot(df["m2hex_y"].dropna(), "C4+-", label="M2Hex Y")
axs[3].set_ylabel("Hexapod Position\n[um]")

for ax in axs:
    ax.grid(":", alpha=0.5)
    ax.legend()

fig.suptitle(f"{test_execution} - M1M3/M2/Hexs/Elevation vs Time")
fig.tight_layout(h_pad=0.3)
fig.patch.set_facecolor('white')   

fig.savefig(f"plots/{test_execution}_m1m3_m2_hexs_el_vs_time_without_aberr.png")
plt.show()

## Gather Data - With Aberrations
Do 4 slews in sequence.
For each slew,

2 seconds before the slew starts, use MTAOS to add some aberrations to the system. The aberrations should correspond to 1um of z4 (focus), z5(astigmatism), z7 (y-coma), and z9 (trefoil), respectively track for 39s, simulating a visit.

Note: the 4 slews need to correspond to elevation angle between 86.5 deg and 82 deg. And, additional margin is needed due to tracking.

In [ ]:
# We start at 85.4 deg because the track fails at 85.5 deg
target_elevations = [85.4, 84.4, 83.4, 82.4]
aberrations = [
    0, # z4 (focus)
    1, # z5 (astigmatism)
    3, # z7 (y-coma)
    5, # z9 (trefoil)
]

await mtcs.rem.mtaos.cmd_resetCorrection.start()
await mtcs.rem.mtaos.cmd_issueCorrection.start(timeout=60.)

t_start_with_aberration = Time.now()
t_start_with_aberration.format = "isot"
print(f"Gathering data - with Aberrations (reset every time) - Start time: {t_start_with_aberration}")

for _el, _ab in zip(target_elevations, aberrations):
    
    print(f"\n\n\n\nCurrent elevation: {_el} - current aberration z{_ab + 4}")

    # the input for addAberration is an array with 19 elements representing 
    # the zernike coefficients starting at z4
    wavefront_errors = np.zeros(19) 
    wavefront_errors[_ab] = 1.0 # um
    await mtcs.rem.mtaos.cmd_addAberration.set_start(wf=wavefront_errors, timeout=10)
    await mtcs.rem.mtaos.cmd_issueCorrection.start(timeout=60.)
    time.sleep(2.)
    
    az, el = await moveMountInElevationSteps(_el, azimuth=190.5)

    target = mtcs.radec_from_azel(az=az, el=el)
    await mtcs.slew_icrs(ra=target.ra, dec=target.dec, rot_type=RotType.Physical, rot=0)

    # 39. seconds is the exposure time (2 * 15 sec) plus readout
    time.sleep(39.)
    await mtcs.stop_tracking()
    
    await mtcs.rem.mtaos.cmd_resetCorrection.start()
    await mtcs.rem.mtaos.cmd_issueCorrection.start(timeout=60.)
    
t_end_with_aberration = Time.now()
t_end_with_aberration.format = "isot"
print(f"Gathering data - with Aberrations (reset every time) - End time: {t_end_with_aberration}")

In [ ]:
print(t_start_with_aberration, t_end_with_aberration)

### Plot Optics vs Time

Plot the following as a function of time during the above process:

- mount elevation
- m1m3 actuator 101 z force
- m2 actuator B1 force
- camera hex y position
- m2 hex y position

In [ ]:
# Query telemetry
df_aberr = await get_data_from_efd(
    exec_info.loc, 
    t_start_with_aberration, 
    t_end_with_aberration)

In [ ]:
fig, axs = plt.subplots(figsize=(10, 10), nrows=4, sharex=True)

axs[0].plot(df_aberr["mount_el"].dropna(), "k", label="Mount Elevation")
axs[0].set_ylabel("Mount El\n[deg]")

axs[1].plot(df_aberr["m1m3_z101"].dropna(), "C0o-", label="z")
axs[1].set_ylabel("M1M3 Forces\n[--]")

# axs[2].plot(df_aberr["m2b1_applied"].dropna(), "C1^-", label="applied")
# axs[2].plot(df_aberr["m2b1_gravLut"].dropna(), "C2v-", label="Gravity LUT")
axs[2].plot(df_aberr["m2b1_measured"].dropna(), "C3-", label="Measured")
axs[2].set_ylabel("M2 Forces\n[--]")

axs[3].plot(df_aberr["camhex_y"].dropna(), "C3x-", label="CamHex Y")
axs[3].plot(df_aberr["m2hex_y"].dropna(), "C4+-", label="M2Hex Y")
axs[3].set_ylabel("Hexapod Position\n[um]")

for ax in axs:
    ax.grid(":", alpha=0.5)
    ax.legend()

fig.suptitle(f"{test_execution} - M1M3/M2/Hexs/Elevation with Aberrations vs Time")
fig.tight_layout(h_pad=0.3)
fig.patch.set_facecolor('white')   

fig.savefig(f"plots/{test_execution}_m1m3_m2_hexs_el_vs_time_with_aberr.png")
plt.show()

## Gather Data - With Accumulated Aberrations

In [ ]:
# We start at 85.4 deg because the track fails at 85.5 deg
target_elevations = [85.4, 84.4, 83.4, 82.4]
aberrations = [
    0, # z4 (focus)
    1, # z5 (astigmatism)
    3, # z7 (y-coma)
    5, # z9 (trefoil)
]

await mtcs.rem.mtaos.cmd_resetCorrection.start()
await mtcs.rem.mtaos.cmd_issueCorrection.start(timeout=60.)

t_start_with_acc_aberration = Time.now()
t_start_with_acc_aberration.format = "isot"
print(f"Gathering data - with Aberrations Accumulated - Start time: {t_start_with_acc_aberration}")

for _el, _ab in zip(target_elevations, aberrations):
    
    print(f"\n\n\n\nCurrent elevation: {_el} - current aberration z{_ab + 4}")

    # the input for addAberration is an array with 19 elements representing 
    # the zernike coefficients starting at z4
    wavefront_errors = np.zeros(19) 
    wavefront_errors[_ab] = 1.0 # um
    await mtcs.rem.mtaos.cmd_addAberration.set_start(wf=wavefront_errors, timeout=10)
    await mtcs.rem.mtaos.cmd_issueCorrection.start(timeout=60.)
    time.sleep(2.)
    
    az, el = await moveMountInElevationSteps(_el, azimuth=190.5)

    target = mtcs.radec_from_azel(az=az, el=el)
    await mtcs.slew_icrs(ra=target.ra, dec=target.dec, rot_type=RotType.Physical, rot=0)

    # 39. seconds is the exposure time (2 * 15 sec) plus readout
    time.sleep(39.)
    await mtcs.stop_tracking()
    
t_end_with_acc_aberration = Time.now()
t_end_with_acc_aberration.format = "isot"
print(f"Gathering data - with Aberrations Accumulated - End time: {t_start_with_acc_aberration}")

In [ ]:
print(f"Gathering data - with Aberrations Accumulated - Start time: {t_start_with_acc_aberration}")
print(f"Gathering data - with Aberrations Accumulated - End time: {t_end_with_acc_aberration}")

### Plot Optics vs Time

In [ ]:
# Query telemetry
df_aberr_acc = await get_data_from_efd(
    exec_info.loc, 
    t_start_with_acc_aberration, 
    t_end_with_acc_aberration)

In [ ]:
fig, axs = plt.subplots(figsize=(10, 10), nrows=4, sharex=True)

axs[0].plot(df_aberr_acc["mount_el"].dropna(), "k", label="Mount Elevation")
axs[0].set_ylabel("Mount El\n[deg]")

axs[1].plot(df_aberr_acc["m1m3_z101"].dropna(), "C0o-", label="z")
axs[1].set_ylabel("M1M3 Forces\n[--]")

# axs[2].plot(df_aberr_acc["m2b1_applied"].dropna(), "C1^-", label="applied")
# axs[2].plot(df_aberr_acc["m2b1_gravLut"].dropna(), "C2v-", label="Gravity LUT")
axs[2].plot(df_aberr_acc["m2b1_measured"].dropna(), "C3-", label="Measured")
axs[2].set_ylabel("M2 Forces\n[--]")

axs[3].plot(df_aberr_acc["camhex_y"].dropna(), "C3x-", label="CamHex Y")
axs[3].plot(df_aberr_acc["m2hex_y"].dropna(), "C4+-", label="M2Hex Y")
axs[3].set_ylabel("Hexapod Position\n[um]")

for ax in axs:
    ax.grid(":", alpha=0.5)
    ax.legend()

fig.suptitle(f"{test_execution} - M1M3/M2/Hexs/Elevation with Aberrations vs Time")
fig.tight_layout(h_pad=0.3)
fig.patch.set_facecolor('white')   

fig.savefig(f"plots/{test_execution}_m1m3_m2_hexs_el_vs_time_with_aberr.png")
plt.show()

In [ ]:
print( mtcs.rem.mtaos.evt_m2Correction.get() )

In [ ]:
print( mtcs.rem.mtaos.evt_m2Correction.get() )
await mtcs.rem.mtaos.cmd_resetCorrection.set_start()
print( mtcs.rem.mtaos.evt_m2Correction.get() )

w = np.zeros(19)
w[0] = 1

await mtcs.rem.mtaos.cmd_addAberration.set_start(wf=w)
print( mtcs.rem.mtaos.evt_m2Correction.get() )

## Compare against the corrections sent by MTAOS. 

check the force and hexapod position differences between steps 10 and 12.

Compare against the corrections sent by MTAOS. They should match.

M1M3 and M2 not using LUT so when no aberrations are applied, all forces are 0. 

To compare forces and hexapod positions, I will find the inPosition flag for the mount and will compare them 3 seconds after it reached position. 

In [ ]:
# Without aberrations
t_start = "2022-06-14T16:24:47.387"
t_start = time.Time(t_start, format="isot", scale="utc")

t_end = "2022-06-14T16:31:25.569"
t_end = time.Time(t_end, format="isot", scale="utc")

In [ ]:
# With aberrations 2022-06-14T16:34:15.195 2022-06-14T16:38:24.966
t_start_aberr = "2022-06-14T16:32:47.387"
t_start_aberr = time.Time(t_start_aberr, format="isot", scale="utc")

t_end_aberr = "2022-06-14T16:38:25.966"
t_end_aberr = time.Time(t_end_aberr, format="isot", scale="utc")

In [ ]:
# Axis in Position in the slews.

In [ ]:
inPosition_without = await client.select_time_series(
    'lsst.sal.MTMount.logevent_elevationInPosition', 
    ['inPosition'], 
    t_start, 
    t_end
)
slews_without = inPosition_without[inPosition_without.inPosition == True]

In [ ]:
startTracking_without = await client.select_time_series(
    'lsst.sal.MTMount.command_startTracking', 
    ['*'], 
    t_start, 
    t_end
)
startTracking_without

In [ ]:
inPosition_withaberr = await client.select_time_series(
    'lsst.sal.MTMount.logevent_elevationInPosition', 
    ['inPosition'], 
    t_start_aberr, 
    t_end_aberr
)
slews_withaberr = inPosition_withaberr[inPosition_withaberr.inPosition == True]

### M1 forces vs MTAOS corrections

In [ ]:
line = []
interval =0.5

for k in np.arange(len(startTracking_without)):

    # Query M1M3 forces 1 second before the slew command is sent
    
    df_m1m3_forces_withaberr = await client.select_time_series(
        "lsst.sal.MTM1M3.forceActuatorData",
        ["zForce101","zForce10", "zForce1"]  ,       
        t_start_aberr, 
        t_end_aberr
    )

    # Query M1M3 forces 2 seconds before correction is issued
    df_m1m3_forces_without = await client.select_time_series(
        "lsst.sal.MTM1M3.forceActuatorData",
        ["zForce101","zForce10", "zForce1"]  ,       
        t_start,
        t_end
    )


    df_mount_el = await client.select_time_series(
        'lsst.sal.MTMount.elevation', 'actualPosition', 
        Time(startTracking_withaberr.index[k])-TimeDelta(1.5, format='sec'), 
        Time(startTracking_withaberr.index[k])-TimeDelta(0.5, format='sec')
    )



    # Build each line 
    line.append(
             {'time': Time(df_m1m3_applied_corrections.index[k]),
              'zForce101_Before': df_m1m3_forces_without.zForce101.mean(),
              'zForce101_After': df_m1m3_forces_withaberr.zForce101.mean(),
              'zForce10_Before': df_m1m3_forces_without.zForce10.mean(),
              'zForce10_After': df_m1m3_forces_withaberr.zForce10.mean(),
              'zForce1_Before': df_m1m3_forces_without.zForce1.mean(),
              'zForce1_After': df_m1m3_forces_withaberr.zForce1.mean(),
              
              'corr_zForce101':df_m1m3_applied_corrections.iloc[k].zForces101,
              'corr_zForce10':df_m1m3_applied_corrections.iloc[k].zForces10,
              'corr_zForce1':df_m1m3_applied_corrections.iloc[k].zForces1,

              'elevation' : df_mount_el.actualPosition.mean()
             }
         )
    

m1m3_aos = pd.DataFrame(line)

In [ ]:
# Without AO- Get the M1 zForce 101 position and elevation angle X seconds after the mount reached position. 

line = []
interval =10

for k in np.arange(len(slews_without)):
    inPos = Time(slews_without.index[k])
    print(inPos)
    t1 = inPos +  TimeDelta(interval, format='sec')
    t2 = t1 +  TimeDelta(1, format='sec')
    
    df_m1m3_101 = await client.select_time_series(
        "lsst.sal.MTM1M3.forceActuatorData",
        "zForce101",
        t1, 
        t2,
    )
    
    df_mount_el = await client.select_time_series(
        'lsst.sal.MTMount.elevation', 'actualPosition', 
        t1, t2)

    line.append(
        {'time': t1,
         'zForce101': df_m1m3_101.zForce101.mean(),
         'elevation' : df_mount_el.actualPosition.mean()
        }
    )
    
df_m1m3_without = pd.DataFrame(line)   

In [ ]:
# Without AO- Get the M1 zForce 101 position and elevation angle X seconds after the mount reached position. 
 
line = []

for k in np.arange(len(slews_withaberr)):
    inPos = Time(slews_withaberr.index[k])
    print(inPos)
    t1 = inPos +  TimeDelta(interval, format='sec')
    t2 = t1 +  TimeDelta(1, format='sec')
    
    df_m1m3_101 = await client.select_time_series(
        "lsst.sal.MTM1M3.forceActuatorData",
        "zForce101",
        t1, 
        t2,
    )
    
    df_mount_el = await client.select_time_series(
        'lsst.sal.MTMount.elevation', 'actualPosition', 
        t1, t2)

    line.append(
        {'time': t1,
         'zForce101': df_m1m3_101.zForce101.mean(),
         'elevation' : df_mount_el.actualPosition.mean()
        }
    )
    
df_m1m3_withaberr = pd.DataFrame(line)   

In [ ]:
m1m3_correction = await client.select_time_series(
    'lsst.sal.MTAOS.logevent_m1m3Correction', 
    [f"zForces101"], 
    t_start_aberr, 
    t_end_aberr
)

In [ ]:
# Print force differences between aberr and nonaberr slews vs MTAOS corrections for each type of aberration
print("1 um of z4 (Focus)")
print(f"zForces101_diff    \t Time ---  \n"
      f"{df_m1m3_withaberr.iloc[0].zForce101-df_m1m3_without.iloc[0].zForce101:0.5f}    \t {df_m1m3_withaberr.iloc[0].time} \n"
      f"zForces101_corr    \t Time  ---- MTAOS correction \n"
      f"{m1m3_correction.iloc[1].zForces101:0.5f} \t {m1m3_correction.index[1]} \n")

print("1 um of z5 (Astigmatism)")
print(f"zForces101_diff    \t Time ---  \n"
      f"{df_m1m3_withaberr.iloc[1].zForce101-df_m1m3_without.iloc[1].zForce101:0.5f}    \t {df_m1m3_withaberr.iloc[1].time} \n"
      f"zForces101_corr    \t Time  ---- MTAOS correction \n"
      f"{m1m3_correction.iloc[3].zForces101:0.5f} \t {m1m3_correction.index[3]} \n")

print("1 um of z7 (y-Coma)")
print(f"zForces101_diff    \t Time ---  \n"
      f"{df_m1m3_withaberr.iloc[2].zForce101-df_m1m3_without.iloc[2].zForce101:0.5f}    \t {df_m1m3_withaberr.iloc[2].time} \n"
      f"zForces101_corr    \t Time  ---- MTAOS correction \n"
      f"{m1m3_correction.iloc[5].zForces101:0.5f} \t {m1m3_correction.index[5]} \n")

print("1 um of z9 (Trefoil)")
print(f"zForces101_diff    \t Time ---  \n"
      f"{df_m1m3_withaberr.iloc[3].zForce101-df_m1m3_without.iloc[3].zForce101:0.5f}    \t {df_m1m3_withaberr.iloc[3].time} \n"
      f"zForces101_corr    \t Time  ---- MTAOS correction \n"
      f"{m1m3_correction.iloc[7].zForces101:0.5f} \t {m1m3_correction.index[7]} \n")

In [ ]:
m1m3_correction_applied = await client.select_time_series(
    'lsst.sal.MTM1M3.command_applyActiveOpticForces', 
    [f"zForces{i}" for i in range(156)], 
    t_start_aberr.utc, 
    t_end_aberr.utc
)

In [ ]:
fig, axs = plt.subplots(2,2, figsize=(16, 8), sharex=True)

axs[0,0].plot(df_m1m3_withaberr.iloc[0].zForce101-m1m3_correction.iloc[1].zForces101, "C0o", label="M1M3 ATAOS - Commanded Applied Forces \n 1 um of z4 (Focus)")
axs[0,0].plot(df_m1m3_without.iloc[0].zForce101, "C0*", label="M1M3 Applied Focus")

# axs[0,1].plot(m1m3_correction.iloc[3].dropna()-m1m3_correction_applied.iloc[3].dropna(), "C0o", label="M1M3 ATAOS - Commanded Applied Forces \n 1 um of z5 (Astigmatism)")
# #axs[0,1].plot(m1m3_correction_applied.iloc[1].dropna(), "C0o", label="M1M3 Applied Focus")

# axs[1,0].plot(m1m3_correction.iloc[5].dropna()-m1m3_correction_applied.iloc[5].dropna(), "C1^-", label="M1M3 ATAOS - Commanded Applied Forces \n 1 um of z7 (y-coma)")
# #axs[1,0].plot(m1m3_correction_applied.iloc[2].dropna(), "C0o", label="M1M3 Applied Focus")

# axs[1,1].plot(m1m3_correction.iloc[7].dropna()-m1m3_correction_applied.iloc[7].dropna(), "C3x-", label="M1M3 ATAOS - Commanded Applied Forces \n 1 um of z9 (Trefoil)")
# #axs[1,1].plot(m1m3_correction_applied.iloc[3].dropna(), "C0o", label="M1M3 Applied Focus")

for ax in axs.flat:
    ax.grid(":", alpha=0.3)
    ax.legend()

fig.suptitle(f"{test_execution} - M1M3 AOS corrections - M1M3 Commanded Apply Forces (10e3 N)")
fig.tight_layout(h_pad=0.3)
fig.patch.set_facecolor('white')   

fig.savefig(f"plots/{test_execution}_m1m3_forces_vs_aos.png")
plt.show()

### M2 forces vs MTAOS corrections

In [ ]:
 # M2 B1?
df_m2 = await client.select_time_series(
        "lsst.sal.MTM2.axialForce",
        fields=[
            "applied0",
        ],
        start=start, 
        end=end,
    )

In [ ]:
m2_correction = await client.select_time_series(
    'lsst.sal.MTAOS.logevent_m2Correction', 
    [f"zForces{i}" for i in range(72)], 
    t_start_aberr, 
    t_end_aberr
)
# Change column names to match that of commanded forces, aka axial. 
m2_correction.columns = m2_correction.columns.str.replace("zForces", "axial")

In [ ]:
m2_correction_applied = await client.select_time_series(
    'lsst.sal.MTM2.command_applyForces', 
    [f"axial{i}" for i in range(72)], 
    t_start_aberr, 
    t_end_aberr)

In [ ]:
fig, axs = plt.subplots(2,2, figsize=(16, 8), sharex=True)

axs[0,0].plot(m2_correction.iloc[1]-m2_correction_applied.iloc[1], "C0*", label="M2 ATAOS - Commanded Applied Forces \n 1 um of z4 (Focus)")
#axs[0,0].plot(m1m3_correction_applied.iloc[0].dropna(), "C0o", label="M1M3 Applied Focus")

axs[0,1].plot(m2_correction.iloc[3]-m2_correction_applied.iloc[3], "C0o", label="M2 ATAOS - Commanded Applied Forces \n 1 um of z5 (Astigmatism)")
#axs[0,1].plot(m1m3_correction_applied.iloc[1].dropna(), "C0o", label="M1M3 Applied Focus")

axs[1,0].plot(m2_correction.iloc[5]-m2_correction_applied.iloc[5], "C1^-", label="M2 ATAOS - Commanded Applied Forces \n 1 um of z7 (y-coma)")
#axs[1,0].plot(m1m3_correction_applied.iloc[2].dropna(), "C0o", label="M1M3 Applied Focus")

axs[1,1].plot(m2_correction.iloc[7]-m2_correction_applied.iloc[7], "C3x-", label="M2 ATAOS - Commanded Applied Forces \n 1 um of z9 (Trefoil)")
#axs[1,1].plot(m1m3_correction_applied.iloc[3].dropna(), "C0o", label="M1M3 Applied Focus")

for ax in axs.flat:
    ax.grid(":", alpha=0.3)
    ax.legend()

fig.suptitle(f"{test_execution} - M2 AOS corrections - M2 Commanded Apply Axial0 Force (10e2 N)")
fig.tight_layout(h_pad=0.3)
fig.patch.set_facecolor('white')   

fig.savefig(f"plots/{test_execution}_m2_with_aberr.png")
plt.show()

### CamHex vs MTAOS corrections

In [ ]:
# Without AO- Get the CamHex position x,y,z,u,v 10 second after the mount reached position, and elevation angle.  

line = []
interval = 10

for k in np.arange(len(slews_without)):
    inPos = Time(slews_without.index[k])
    print(inPos)
    t1 = inPos +  TimeDelta(interval, format='sec')
    t2 = t1 +  TimeDelta(1, format='sec')
    
    df_camhex = await client.select_time_series(
        "lsst.sal.MTHexapod.application",
        ["position0", "position1", "position2", "position3", "position4", "position5"],
        t1, 
        t2,
        index=2
    )
    
    df_mount_el = await client.select_time_series(
        'lsst.sal.MTMount.elevation', 'actualPosition', 
        t1, t2)

    line.append(
        {'time': t1,
         'x': df_camhex.position0.mean(),
         'y': df_camhex.position1.mean(),
         'z': df_camhex.position2.mean(), 
         'u': df_camhex.position3.mean(),  
         'v': df_camhex.position4.mean(),
         'w': df_camhex.position5.mean(),
         'elevation' : df_mount_el.actualPosition.mean()
        }
    )
    
df_camhex_without = pd.DataFrame(line)   

In [ ]:
# With Aberrations- Get the CamHex position x,y,z,u,v 5 seconds after the mount reached position, and elevation angle.  

line = []

for k in np.arange(len(slews_withaberr)):
    inPos = Time(slews_withaberr.index[k])
    print(inPos)
    t1 = inPos +  TimeDelta(interval, format='sec')
    t2 = t1 +  TimeDelta(1, format='sec')
    
    df_camhex = await client.select_time_series(
        "lsst.sal.MTHexapod.application",
        ["position0", "position1", "position2", "position3", "position4", "position5"],
        t1, 
        t2,
        index=2
    )
    
    df_mount_el = await client.select_time_series(
        'lsst.sal.MTMount.elevation', 'actualPosition', 
        t1, t2)

    line.append(
        {'time': t1,
         'x': df_camhex.position0.mean(),
         'y': df_camhex.position1.mean(),
         'z': df_camhex.position2.mean(), 
         'u': df_camhex.position3.mean(),  
         'v': df_camhex.position4.mean(),
         'w': df_camhex.position5.mean(),
         'elevation' : df_mount_el.actualPosition.mean()
        }
    )
    
df_camhex_withaberr = pd.DataFrame(line)   

In [ ]:
# Get the MTAOS corrections 
cam_hexapod_correction_applied_xyz = await client.select_time_series(
    'lsst.sal.MTHexapod.logevent_uncompensatedPosition', 
    ["x", "y", "z", "MTHexapodID"], 
    t_start_aberr, 
    t_end_aberr,
    index=1
)

cam_hexapod_correction_applied_uv = await client.select_time_series(
    'lsst.sal.MTHexapod.logevent_uncompensatedPosition', 
    ["u", "v", "MTHexapodID"], 
    t_start_aberr, 
    t_end_aberr,
    index=1
)

In [ ]:
cam_hexapod_correction_command_xyz = await client.select_time_series(
    'lsst.sal.MTHexapod.command_move', 
    ["x", "y", "z", "MTHexapodID"], 
    t_start_aberr, 
    t_end_aberr,
    index=1
)

cam_hexapod_correction_command_uv = await client.select_time_series(
    'lsst.sal.MTHexapod.command_move', 
    ["u", "v", "MTHexapodID"], 
    t_start_aberr, 
    t_end_aberr,
    index=1
)

In [ ]:
# Print camhex position differences vs MTAOS corrections for each type of aberration
print("1 um of z4 (Focus)")
print(f"x_diff    \t y_diff    \t z_diff    \t Time \n"
      f"{df_camhex_withaberr.iloc[0].x-df_camhex_without.iloc[0].x:0.5f} \t {df_camhex_withaberr.iloc[0].y-df_camhex_without.iloc[0].y:0.5f} \t"
      f" {df_camhex_withaberr.iloc[0].z-df_camhex_without.iloc[0].z:0.5f} \t {df_camhex_withaberr.iloc[0].time} \n"
      f"x_corr    \t y_corr    \t z_corr    \t Time  ---- MTAOS correction \n"
      f"{cam_hexapod_correction_applied_xyz.iloc[0].x:0.5f} \t {cam_hexapod_correction_applied_xyz.iloc[0].y:0.5f} \t {cam_hexapod_correction_applied_xyz.iloc[0].z:0.5f} \t"
      f"{cam_hexapod_correction_applied_xyz.index[0]} \n")

print("1 um of z5 (Astigmatism)")
print(f"x_diff    \t y_diff    \t z_diff    \t Time  \n"
      f"{df_camhex_withaberr.iloc[1].x-df_camhex_without.iloc[1].x:0.5f} \t {df_camhex_withaberr.iloc[1].y-df_camhex_without.iloc[1].y:0.5f} \t"
      f" {df_camhex_withaberr.iloc[1].z-df_camhex_without.iloc[1].z:0.5f} \t {df_camhex_withaberr.iloc[1].time} \n"
      f"x_corr    \t y_corr    \t z_corr    \t Time  ---- MTAOS correction \n"
      f"{cam_hexapod_correction_applied_xyz.iloc[2].x:0.5f} \t {cam_hexapod_correction_applied_xyz.iloc[2].y:0.5f} \t {cam_hexapod_correction_applied_xyz.iloc[2].z:0.5f} \t"
      f"{cam_hexapod_correction_applied_xyz.index[2]} \n")

print("1 um of z7 (y-Coma)")
print(f"x_diff    \t y_diff    \t z_diff    \t Time  \n"
      f"{df_camhex_withaberr.iloc[2].x-df_camhex_without.iloc[2].x:0.5f} \t {df_camhex_withaberr.iloc[2].y-df_camhex_without.iloc[2].y:0.5f} \t"
      f" {df_camhex_withaberr.iloc[2].z-df_camhex_without.iloc[2].z:0.5f} \t {df_camhex_withaberr.iloc[2].time} \n"
      f"x_corr    \t y_corr    \t z_corr    \t Time  ---- MTAOS correction \n"
      f"{cam_hexapod_correction_applied_xyz.iloc[4].x:0.5f} \t {cam_hexapod_correction_applied_xyz.iloc[4].y:0.5f} \t {cam_hexapod_correction_applied_xyz.iloc[4].z:0.5f} \t"
      f"{cam_hexapod_correction_applied_xyz.index[4]} \n")

print("1 um of z9 (Trefoil)")
print(f"x_diff    \t y_diff    \t z_diff    \t Time \n"
      f"{df_camhex_withaberr.iloc[3].x-df_camhex_without.iloc[3].x:0.5f} \t {df_camhex_withaberr.iloc[3].y-df_camhex_without.iloc[3].y:0.5f} \t"
      f" {df_camhex_withaberr.iloc[3].z-df_camhex_without.iloc[3].z:0.5f} \t {df_camhex_withaberr.iloc[3].time} \n"
      f"x_corr    \t y_corr    \t z_corr    \t Time  ---- MTAOS correction \n"
      f"{cam_hexapod_correction_applied_xyz.iloc[6].x:0.5f} \t {cam_hexapod_correction_applied_xyz.iloc[6].y:0.5f} \t {cam_hexapod_correction_applied_xyz.iloc[6].z:0.5f} \t"
      f"{cam_hexapod_correction_applied_xyz.index[6]} \n")

In [ ]:
# It's difficult to compare because MTAOS and lsst.sal.MTHexapod.logevent_uncompensatedPosition are published once a second
cam_hexapod_correction_command_xyz
cam_hexapod_correction_applied_xyz

### M2Hex vs MTAOS corrections

In [ ]:
m2_hexapod_correction_computed_xyz = await client.select_time_series(
    'lsst.sal.MTAOS.logevent_m2HexapodCorrection', 
    ["x", "y", "z"], 
    start.utc, 
    end.utc
)

m2_hexapod_correction_computed_uv = await client.select_time_series(
    'lsst.sal.MTAOS.logevent_m2HexapodCorrection', 
    ["u", "v"], 
    start.utc, 
    end.utc
)

In [ ]:
m2_hexapod_correction_applied_xyz = await client.select_time_series(
    'lsst.sal.MTHexapod.logevent_uncompensatedPosition', 
    ["x", "y", "z", "MTHexapodID"], 
    start.utc, 
    end.utc,
    index=2
)

m2_hexapod_correction_applied_uv = await client.select_time_series(
    'lsst.sal.MTHexapod.logevent_uncompensatedPosition', 
    ["u", "v", "MTHexapodID"], 
    start.utc, 
    end.utc,
    index=2
)

In [ ]:
m2_hexapod_correction_command_xyz = await client.select_time_series(
    'lsst.sal.MTHexapod.command_move', 
    ["x", "y", "z", "MTHexapodID"], 
    start.utc, 
    end.utc,
    index=2
)

m2_hexapod_correction_command_uv = await client.select_time_series(
    'lsst.sal.MTHexapod.command_move', 
    ["u", "v", "MTHexapodID"], 
    start.utc, 
    end.utc,
    index=2
)

## Wrap up
lower m1m3,
put hardware to OFFLINE
put all simulators to STANDBY
done.

In [ ]:
# Put the telescope back to the original position
# Specially if running at TTS
await mtcs.point_azel(az=0, el=80)

In [ ]:
await mtcs.lower_m1m3()

In [ ]:
await mtcs.set_state(state=salobj.State.STANDBY, components=["mtm1m3"])

In [ ]:
await mtcs.set_state(state=salobj.State.STANDBY, components=["mtaos"])

In [ ]:
await mtcs.set_state(state=salobj.State.STANDBY, components=["mtm2"])

In [ ]:
await mtcs.set_state(state=salobj.State.STANDBY, components=["mthexapod_1"])

In [ ]:
await mtcs.set_state(state=salobj.State.STANDBY, components=["mthexapod_2"])

In [ ]:
if exec_info.loc == "summit":
    await mtcs.standby()

else:
    # Bring the system back to the original state
    await mtcs.set_state(
        state=salobj.State.ENABLED,
        components=[
            "mtm1m3", 
            "mtm2",
            "mthexapod_1",
            "mthexapod_2",
            "mtaos",
        ],
        overrides={
            "mtm1m3": "Default"
        }
    )